In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
DATA_DIR = '../../../In-situ Meas Data/Build Command Data/XYPT Commands/' # Directory of XYPT files

dt = 1E-5 # 10 microseconds

In [41]:
data = pd.read_csv(DATA_DIR + 'T500_3D_Scan_Strategies_fused_layer0002.csv', header=None, names=['X', 'Y', 'P', 'T', 'deprecated'])

In [42]:
trigger_indx = np.where(np.array(data['T']))[0]
X = np.array(data['X'])
Y = np.array(data['Y'])
P = np.array(data['P'])

In [6]:
# Speed
x_vel = np.zeros(X.shape)
x_vel[1:-1] = (X[2:]-X[:-2]) / 2 / dt

y_vel = np.zeros(X.shape)
y_vel[1:-1] = (Y[2:]-Y[:-2]) / 2 / dt

speed = np.sqrt(np.square(x_vel) + np.square(y_vel))

In [7]:
# Direction
vel_dir_x = x_vel / (speed + 1e-8)
vel_dir_y = y_vel / (speed + 1e-8)

In [8]:
# Acceleration
x_accel = np.zeros(X.shape)
x_accel[1:-1] = (x_vel[2:]-x_vel[:-2]) / 2 / dt

y_accel = np.zeros(Y.shape)
y_accel[1:-1] = (y_vel[2:]-y_vel[:-2]) / 2 / dt

accel = np.sqrt(np.square(x_accel) + np.square(y_accel))
accel[np.where(speed[1:] < speed[:-1])[0]+1] = accel[np.where(speed[1:] < speed[:-1])[0]+1]*-1

In [9]:
# Energy Density
en_den = P / (speed + 1e-8)

In [10]:
# Heating/Cooling
heating = np.zeros(X.shape)
heating[np.where(P[1:]>P[:-1])[0]+1] = 1
heating[np.where(P[1:]<P[:-1])[0]+1] = -1

In [126]:
# Turning
turn_threshold = 0.01 # Tune me

turning = np.zeros(X.shape)
turn1 = np.zeros(X.shape)
turn2 = np.zeros(X.shape)

turn1[1:] = np.abs(vel_dir_x[1:]-vel_dir_x[:-1])
turn2[1:] = np.abs(vel_dir_y[1:]-vel_dir_y[:-1])
turning = np.any([(turn1 > turn_threshold), (turn2 > turn_threshold)], axis=0)
turning = turning.astype('int')

array([0, 0, 0, ..., 0, 0, 1])

In [114]:
# Residual Heat
R = 10 # parts are 10 mm x 10 mm
T = 5
max_iters = round(T/dt)

residual_heat = np.zeros(X.shape[0])
for i in tqdm(trigger_indx):
    d_ij = np.sqrt(np.square(X[i]-X[max(0, i-max_iters):i]) + np.square(Y[i]-Y[max(0, i-max_iters):i]))
    t_ij = np.linspace((min(max_iters, i))*dt, dt, min(max_iters, i))
    p_j = P[max(0, i-max_iters):i]
    residual_heat[i] += np.sum(np.square((R-d_ij)/R) * ((T-t_ij)/T) * p_j)

100%|██████████████████████████████████████████████████████████████████████████████| 2927/2927 [00:37<00:00, 79.06it/s]


In [128]:
# Merge together, normalize, and save to an excel file
P_trig = P[trigger_indx]
speed_trig = speed[trigger_indx]
x_dir_trig = vel_dir_x[trigger_indx]
y_dir_trig = vel_dir_y[trigger_indx]
accel_trig = accel[trigger_indx]
en_den_trig = en_den[trigger_indx]
heating_trig = heating[trigger_indx]
turning_trig = turning[trigger_indx]
residual_heat_trig = residual_heat[trigger_indx]

pp = pd.DataFrame({'P': P_trig, 'speed': speed_trig, 'x_dir': x_dir_trig, 'y_dir': y_dir_trig, 'accel': accel_trig,
                  'ed': en_den_trig, 'heating': heating_trig, 'turning': turning_trig, 'res_heat': residual_heat_trig})



# pp_normalized = pd.DataFrame()


In [ ]:
# Merge with the labels excel files
label = pd.read_excel()


### Now cycle through each layer 

In [135]:
pp_dict = dict()

for ii in tqdm(range(2,251)):
    if ii < 10:
        layer_text = 'layer000' + str(ii)
    elif ii < 100:
        layer_text = 'layer00' + str(ii)
    else:
        layer_text = 'layer0' + str(ii)
        
    data = pd.read_csv(DATA_DIR + 'T500_3D_Scan_Strategies_fused_' + layer_text + '.csv', header=None, 
                       names=['X', 'Y', 'P', 'T', 'deprecated'])
    
    trigger_indx = np.where(np.array(data['T']))[0]
    X = np.array(data['X'])
    Y = np.array(data['Y'])
    P = np.array(data['P'])
    
    # Speed
    x_vel = np.zeros(X.shape)
    x_vel[1:-1] = (X[2:]-X[:-2]) / 2 / dt

    y_vel = np.zeros(X.shape)
    y_vel[1:-1] = (Y[2:]-Y[:-2]) / 2 / dt

    speed = np.sqrt(np.square(x_vel) + np.square(y_vel))
    
    # Direction
    vel_dir_x = x_vel / (speed + 1e-8)
    vel_dir_y = y_vel / (speed + 1e-8)
    
    # Acceleration
    x_accel = np.zeros(X.shape)
    x_accel[1:-1] = (x_vel[2:]-x_vel[:-2]) / 2 / dt

    y_accel = np.zeros(Y.shape)
    y_accel[1:-1] = (y_vel[2:]-y_vel[:-2]) / 2 / dt

    accel = np.sqrt(np.square(x_accel) + np.square(y_accel))
    accel[np.where(speed[1:] < speed[:-1])[0]+1] = accel[np.where(speed[1:] < speed[:-1])[0]+1]*-1
    
    # Energy Density
    en_den = P / (speed + 1e-8)
    
    # Heating/Cooling
    heating = np.zeros(X.shape)
    heating[np.where(P[1:]>P[:-1])[0]+1] = 1
    heating[np.where(P[1:]<P[:-1])[0]+1] = -1
    
    # Turning
    turn_threshold = 0.01 # Tune me

    turning = np.zeros(X.shape)
    turn1 = np.zeros(X.shape)
    turn2 = np.zeros(X.shape)

    turn1[1:] = np.abs(vel_dir_x[1:]-vel_dir_x[:-1])
    turn2[1:] = np.abs(vel_dir_y[1:]-vel_dir_y[:-1])
    turning = np.any([(turn1 > turn_threshold), (turn2 > turn_threshold)], axis=0)
    turning = turning.astype('int')
    
    # Residual Heat
    R = 10 # parts are 10 mm x 10 mm
    T = 5
    max_iters = round(T/dt)

    residual_heat = np.zeros(X.shape[0])
    for i in trigger_indx:
        d_ij = np.sqrt(np.square(X[i]-X[max(0, i-max_iters):i]) + np.square(Y[i]-Y[max(0, i-max_iters):i]))
        t_ij = np.linspace((min(max_iters, i))*dt, dt, min(max_iters, i))
        p_j = P[max(0, i-max_iters):i]
        residual_heat[i] += np.sum(np.square((R-d_ij)/R) * ((T-t_ij)/T) * p_j)
        
    P_trig = P[trigger_indx]
    speed_trig = speed[trigger_indx]
    x_dir_trig = vel_dir_x[trigger_indx]
    y_dir_trig = vel_dir_y[trigger_indx]
    accel_trig = accel[trigger_indx]
    en_den_trig = en_den[trigger_indx]
    heating_trig = heating[trigger_indx]
    turning_trig = turning[trigger_indx]
    residual_heat_trig = residual_heat[trigger_indx]

    pp = pd.DataFrame({'P': P_trig, 'speed': speed_trig, 'x_dir': x_dir_trig, 'y_dir': y_dir_trig, 'accel': accel_trig,
                      'ed': en_den_trig, 'heating': heating_trig, 'turning': turning_trig, 'res_heat': residual_heat_trig})
    
    pp_dict[ii] = pp

  0%|▎                                                                              | 1/249 [02:10<8:58:56, 130.39s/it]


KeyboardInterrupt: 